In [1]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments
import pandas as pd
import torch
import re
import os
from convokit import Corpus, download
from transformers import GPT2Tokenizer
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

# Local save path for corpus & models
save_path = '/Users/halladaykinsey/Desktop/Conversational_Chatbot/conversational_data'
if not os.path.exists(save_path):
    os.makedirs(save_path)

/Users/halladaykinsey/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Downloading & loading the corpus
corpus = Corpus(filename=download("movie-corpus"))

# Extracting conversations into a list
conversations = list(corpus.iter_conversations())

# Saving conversations to a file for later use
pd.DataFrame(conversations).to_csv(os.path.join(save_path, 'conversations.csv'), index=False)

# Printing a sample conversation for verification
for utt_id in conversations[0].get_utterance_ids():
    print(corpus.get_utterance(utt_id).text)

They do not!
They do to!


In [4]:
# Function to clean text by removing brackets, punctuation, and extra spaces
def clean_text(text):
    text = re.sub(r'\[.*?\]', '', text)  
    text = re.sub(r'[^a-zA-Z\s]', '', text)  
    text = re.sub(r'\s+', ' ', text).strip() 
    return text.lower() 

# Cleaning text for all utterances
total_convos = len(conversations)
for i, convo in enumerate(conversations):
    for utt_id in convo.get_utterance_ids():
        utt = corpus.get_utterance(utt_id)
        utt.text = clean_text(utt.text)
    if i % 100 == 0:  # Printing progress every 100 conversations
        print(f"Cleaned {i}/{total_convos} conversations")

Cleaned 0/83097 conversations
Cleaned 100/83097 conversations
Cleaned 200/83097 conversations
Cleaned 300/83097 conversations
Cleaned 400/83097 conversations
Cleaned 500/83097 conversations
Cleaned 600/83097 conversations
Cleaned 700/83097 conversations
Cleaned 800/83097 conversations
Cleaned 900/83097 conversations
Cleaned 1000/83097 conversations
Cleaned 1100/83097 conversations
Cleaned 1200/83097 conversations
Cleaned 1300/83097 conversations
Cleaned 1400/83097 conversations
Cleaned 1500/83097 conversations
Cleaned 1600/83097 conversations
Cleaned 1700/83097 conversations
Cleaned 1800/83097 conversations
Cleaned 1900/83097 conversations
Cleaned 2000/83097 conversations
Cleaned 2100/83097 conversations
Cleaned 2200/83097 conversations
Cleaned 2300/83097 conversations
Cleaned 2400/83097 conversations
Cleaned 2500/83097 conversations
Cleaned 2600/83097 conversations
Cleaned 2700/83097 conversations
Cleaned 2800/83097 conversations
Cleaned 2900/83097 conversations
Cleaned 3000/83097 con

In [5]:
# Initializing an empty list to store conversation data
conversation_data = []

# Looping through processed conversations and collecting text
for convo in conversations:
    for utt_id in convo.get_utterance_ids():
        utt = corpus.get_utterance(utt_id)
        processed_text = utt.text
        conversation_data.append({'text': processed_text})

# Converting conversation data to a DataFrame
df = pd.DataFrame(conversation_data)

# Displaying first few rows of the DataFrame
print(df.head())

          text
0  they do not
1   they do to
2    i hope so
3     she okay
4      lets go


In [6]:
# Creating dictionary to store conversation data
conversation_dfs = {}
conversation_dfs['all_conversations'] = df

# Taking a random sample of 10,000 rows 
subset_size = 10000 
df_subset = df.sample(n=subset_size, random_state=42)

# Verifying size and first few rows of subset
print(df_subset.shape)
print(df_subset.head())

# Splitting the dataset into training and evaluation sets (90% train, 10% eval)
train_df, eval_df = train_test_split(df_subset, test_size=0.1, random_state=42)

(10000, 1)
                                                     text
63127                                 my mom wasnt a goat
134803  i must protect my interests ms kyle and intere...
143713  you said bad things hurt places so maybe good ...
216087                             she hates all freshmen
165209                              what are you gonna do


In [7]:
# Initializing tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Define a function to tokenize input and response texts
def tokenize_text(text, max_length=128):
    return tokenizer.encode(text, truncation=True, padding='max_length', max_length=max_length)

# Tokenizing input and response text for training set
train_df['input_ids'] = train_df['text'].apply(lambda x: tokenize_text(x))
train_df['response_ids'] = train_df['text'].shift(-1).apply(lambda x: tokenize_text(x) if pd.notna(x) else [tokenizer.eos_token_id])

# Tokenizing input and response text for evaluation set
eval_df['input_ids'] = eval_df['text'].apply(lambda x: tokenize_text(x))
eval_df['response_ids'] = eval_df['text'].shift(-1).apply(lambda x: tokenize_text(x) if pd.notna(x) else [tokenizer.eos_token_id])

/Users/halladaykinsey/myenv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
class MovieDialoguesDataset(Dataset):
    def __init__(self, dataframe):
        self.input_ids = dataframe['input_ids'].tolist()
        self.labels = dataframe['response_ids'].tolist()

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [9]:
# Filtering out rows where input_ids or response_ids are not of length 128
train_df = train_df[(train_df['input_ids'].apply(len) == 128) & (train_df['response_ids'].apply(len) == 128)]
eval_df = eval_df[(eval_df['input_ids'].apply(len) == 128) & (eval_df['response_ids'].apply(len) == 128)]

In [10]:
# Create dataset instances
train_dataset = MovieDialoguesDataset(train_df)
eval_dataset = MovieDialoguesDataset(eval_df)

# Checking dataset instances
print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

Train dataset size: 8981
Eval dataset size: 995


In [11]:
# Check for input_ids and response_ids lengths
print("Input IDs lengths:", train_df['input_ids'].apply(len).value_counts())
print("Response IDs lengths:", train_df['response_ids'].apply(len).value_counts())

Input IDs lengths: input_ids
128    8981
Name: count, dtype: int64
Response IDs lengths: response_ids
128    8981
Name: count, dtype: int64


In [12]:
# Global variable to store the trainer instance
trainer = None

def train_model_for_conversation(dataset):
    global trainer  # Declare that we want to use the global trainer variable

    # Loading GPT-2
    model = GPT2LMHeadModel.from_pretrained('gpt2')

    # Defining training arguments for conversation
    training_args = TrainingArguments(
        output_dir=os.path.join(save_path, 'model_results'), 
        num_train_epochs=3, 
        per_device_train_batch_size=2,  
        per_device_eval_batch_size=2,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=os.path.join(save_path, 'logs'),  
        logging_steps=10,
        evaluation_strategy="steps",  # Change to "steps"
        save_strategy="steps",         # Ensure saving strategy is also "steps"
        load_best_model_at_end=True,  # Add this line
    )

    # Creating trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=eval_dataset  # Ensure you pass the eval dataset
    )

    # Training model
    trainer.train()

    # Saving trained model to local path
    model.save_pretrained(os.path.join(save_path, 'trained_conversational_model'))
    tokenizer.save_pretrained(os.path.join(save_path, 'trained_conversational_model'))  # Save tokenizer

    return trainer


In [13]:
# Training model
train_model_for_conversation(train_dataset)

/Users/halladaykinsey/myenv/lib/python3.12/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 10/13473 [00:12<4:12:40,  1.13s/it]

{'loss': 12.7561, 'grad_norm': 225.73989868164062, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


                                                    
  0%|          | 10/13473 [01:41<4:12:40,  1.13s/it]

{'eval_loss': 13.287164688110352, 'eval_runtime': 88.4631, 'eval_samples_per_second': 11.248, 'eval_steps_per_second': 5.629, 'epoch': 0.0}


  0%|          | 20/13473 [01:51<7:25:12,  1.99s/it]  

...

                                                         
 33%|███▎      | 4390/13473 [12:15:39<8:46:10,  3.48s/it]

{'eval_loss': 0.7116278409957886, 'eval_runtime': 113.9823, 'eval_samples_per_second': 8.729, 'eval_steps_per_second': 4.369, 'epoch': 0.98}


 33%|███▎      | 4400/13473 [12:15:47<5:36:57,  2.23s/it] 

{'loss': 0.9422, 'grad_norm': 0.3089836835861206, 'learning_rate': 3.496878131503893e-05, 'epoch': 0.98}


KeyboardInterrupt: 

In [13]:
# Evaluating the model
if trainer is not None:  # Check if trainer is defined
    eval_results = trainer.evaluate()

    # Print metrics in a structured format
    print("Evaluation Metrics:")
    print("=====================")
    for key, value in eval_results.items():
        print(f"{key}: {value:.4f}")
else:
    print("Trainer is not defined.")

100%|██████████| 498/498 [01:34<00:00,  5.25it/s]

Evaluation Metrics:
eval_loss: 0.7101
eval_runtime: 95.4551
eval_samples_per_second: 10.4240
eval_steps_per_second: 5.2170
epoch: 3.0000


In [20]:
# Load the model and tokenizer from the saved path
model_path = '/Users/halladaykinsey/Desktop/Conversational_Chatbot/trained_conversational_model'
model = GPT2LMHeadModel.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()

def generate_response(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    attention_mask = (input_ids != tokenizer.pad_token_id).long()  # Adjust this line to create attention mask

    with torch.no_grad():
        output = model.generate(input_ids, 
                                attention_mask=attention_mask, 
                                max_length=150, 
                                num_return_sequences=1,
                                temperature=0.9,   # Adjust temperature
                                top_k=50,          # Adjust top-k
                                top_p=0.95)        # Adjust top-p

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

conversation_history = ""

def chat():
    global conversation_history
    print("Welcome to the chatbot! Type 'exit' to end the conversation.")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        
        # Append user input to the conversation history
        conversation_history += f"You: {user_input}\n"
        
        # Generate response based on the updated conversation history
        response = generate_response(conversation_history)
        
        print(f"Bot: {response}")
        
        # Append the bot's response to the conversation history
        conversation_history += f"Bot: {response}\n"

In [22]:
# Start chatting
chat()

Welcome to the chatbot! Type 'exit' to end the conversation.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Bot: You: Hello
Bot: You: Hello

You: How are you?
Bot: You: Hello
Bot: You: Hello

You: How are you?

You: If I lived in Los Angeles, what movie would you recommend watching on a cold and stormy Saturday night?



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


ValueError: Input length of input_ids is 157, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.